<a href="https://colab.research.google.com/github/tommymmcguire/IA2_590/blob/main/getinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 26.0 MB/s eta 0:00:00


In [10]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [11]:
def chunk_text(text, chunk_size=500):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]


In [12]:
import sqlite3

def setup_database(db_path='rag_database.db'):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS text_chunks
                 (id INTEGER PRIMARY KEY, chunk TEXT)''')
    conn.commit()
    conn.close()


In [13]:
def insert_chunks_to_database(chunks, db_path='rag_database.db'):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    for chunk in chunks:
        c.execute("INSERT INTO text_chunks (chunk) VALUES (?)", (chunk,))
    conn.commit()
    conn.close()


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

def search_database(query, db_path='rag_database.db', top_k=5):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute("SELECT id, chunk FROM text_chunks")
    results = c.fetchall()

    query_embedding = model.encode([query])[0]
    scores = []

    for id, chunk in results:
        chunk_embedding = model.encode([chunk])[0]
        score = np.dot(query_embedding, chunk_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(chunk_embedding))
        scores.append((score, id, chunk))

    scores.sort(reverse=True)
    return scores[:top_k]


In [ ]:
import openai

openai.api_key = 'your-api-key'

def generate_response_with_context(query, context_chunks):
    prompt = f"{query}\n\n{' '.join(context_chunks)}"
    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=100,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    return response.choices[0].text.strip()


In [20]:
import os

# Step 1: Setup the database
db_path = 'rag_database.db'
setup_database(db_path)

# Step 2: Process PDFs and insert chunks into the database
pdf_directory = './pdfs'
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text, chunk_size=500)  # Adjust chunk_size as needed
    insert_chunks_to_database(chunks, db_path)

print("Finished processing PDFs and populating the database.")

# # Example query for demonstration
# query = "What is the impact of climate change on polar bears?"

# # Step 3: Perform semantic search to find relevant text chunks
# relevant_chunks = search_database(query, db_path, top_k=5)
# context_chunks = [chunk for chunk in relevant_chunks]

# # Step 4: Generate a response using LLM integration with the context
# # This is a simplified example; you'll need to adapt it for your actual LLM API.
# response = generate_response_with_context(query, context_chunks)
# print("Generated Response:", response)


Streaming output truncated to the last 5000 lines.
cover, jacket, shirt, or bed ticks, bags, and sacks (Figure B-18).  Poles can
be improvised from strong branches, tent supports, skis, lengths of pipe or
other objects.  If objects for improvising poles are not available, a blanket,
poncho, or similar item can be rolled from both sides toward the center so the
rolls can be gripped for carrying a patient.  Most flat-surface objects of
suitable size can be used as litters.  Such objects include doors, boards,
window shutters, benches, ladders, cots, and chairs.  If possible, these objects
should be padded for the casualty’s comfort.
(1)
To improvise a litter using a blanket and poles (Figure
B-21), the following steps should be used.
Figure B-21.  Litter made with blanket and poles.
(a)
Open the blanket and lay one pole lengthwise
across the center; then fold the blanket over the pole.
(b)
Place the second pole across the center of the
folded blanket.
B-27
FM 4-25.11/NTRP 4-02.1/AFMAN 44

# New Section